# Postprocess

In [3]:
import pandas as pd
import numpy as np

# Log loss

In [4]:
sample_sub = "../submissions/sampleSubmission.csv"
sample_sub_df = pd.read_csv(sample_sub)

def normalize(row, epsilon=1e-15):
    
    row = row / np.sum(row)
    row = np.maximum(epsilon, row)
    row = np.minimum(1 - epsilon, row)
    
    return row
    
def logloss_mc(y_true, y_probs):
    
    # Normalize probability data frame
    y_probs = y_probs.apply(normalize, axis=1)
        
    log_vals = []
        
    for i, y in enumerate(y_true):
        c = int(y.split("_")[1])
        log_vals.append(- np.log(y_probs.iloc[i,c - 1]))
        
    return np.mean(log_vals)

# Perform cross

In [3]:
df_holdout = pd.read_csv("../data/holdout20.csv")
y_valid = df_holdout.target

In [135]:
sub = pd.read_csv("../submission.csv")

In [75]:
def polish(x, t1=0.95, t1f=1,
              t2=0.9, t2f=1,
              t3=0.85, t3f=1,
              t4=0.8, t4f=1,
              t5=0.75, t5f=1,
              t6=0.7, t6f=1,
              t7=0.65, t7f=1,
              t8=0.6, t8f=1,
              t9=0.55, t9f=1,
              t10=0.50, t10f=1,
              t11=0.45, t11f=1,
              t12=0.40, t12f=1,
              t13=0.35, t13f=1,
              t14=0.30, t14f=1,
              t15=0.25, t15f=1,
              t16=0.20, t16f=1,
              t17=0.15, t17f=1,
              t18=0.10, t18f=1,
              t19=0.05, t19f=1,):
    if x > t1:
        return t1f * x
    elif x > t2:
        return t2f * x
    elif x > t3:
        return t3f * x
    elif x > t4:
        return t4f * x
    elif x > t5:
        return t5f * x
    elif x > t6:
        return t6f * x
    elif x > t7:
        return t7f * x
    elif x > t8:
        return t8f * x
    elif x > t9:
        return t9f * x
    elif x > t10:
        return t10f * x
    elif x > t11:
        return t11f * x
    elif x > t12:
        return t12f * x
    elif x > t13:
        return t13f * x
    elif x > t14:
        return t14f * x
    elif x > t15:
        return t15f * x
    elif x > t16:
        return t16f * x
    elif x > t17:
        return t7f * x
    elif x > t18:
        return t18f * x
    elif x > t19:
        return t19f * x
    else:
        return x
    


In [91]:
def polish_helper(x, threshold, delta, factor):
    if abs(x - threshold) < delta:
        return x * factor
    else:
        return x


def polish_nice(low=0.5,
                high=1.5,
                steps_values=21,
                start=0.00,
                stop=0.20,
                steps_space=21):
    
    # Threshold factors (tfs)
    thresholdfactors = np.linspace(low, high, steps_values)
    
    # Thresholds
    thresholds = np.linspace(start, stop, steps_space)
    
    for threshold in thresholds:
        print "threshold is:", threshold
        values = []
        for factor in thresholdfactors:

            out = pd.concat([sub.id, sub.iloc[:, 1:]
                             .applymap(functools.
                                       partial(polish_helper,
                                               threshold=threshold,
                                               delta=(thresholdfactors[1] - thresholdfactors[0]) / 2,
                                               factor=factor))],
                            axis = 1)

            out.to_csv("../sub_polished.csv", index=False)


            subpolished = pd.read_csv("../sub_polished.csv").iloc[:, 1:]
            ll = logloss_mc(y_valid, subpolished)

            values.append(ll)

        print "value min", np.min(values)
        print "best tf", thresholdfactors[np.argmin(values)]

In [92]:
polish_nice()

threshold is: 0.0
value min 0.466758252597
best tf 0.75
threshold is: 0.01
value min 0.466565898206
best tf 0.75
threshold is: 0.02
value min 0.466854450827
best tf 0.8
threshold is: 0.03
value min 0.467189490233
best tf 0.9
threshold is: 0.04
value min 0.467369589664
best tf 1.0
threshold is: 0.05
value min 0.467338120658
best tf 1.05
threshold is: 0.06
value min 0.467183238965
best tf 1.15
threshold is: 0.07
value min 0.46727237363
best tf 1.1
threshold is: 0.08
value min 0.467273258297
best tf 1.1
threshold is: 0.09
value min 0.467273965208
best tf 1.1
threshold is: 0.1
value min 0.467283610758
best tf 1.1
threshold is: 0.11
value min 0.467253770468
best tf 1.15
threshold is: 0.12
value min 0.46699557185
best tf 1.25
threshold is: 0.13
value min 0.466931774757
best tf 1.25
threshold is: 0.14
value min 0.466925471624
best tf 1.25
threshold is: 0.15
value min 0.467038369582
best tf 1.25
threshold is: 0.16
value min 0.466991999754
best tf 1.25
threshold is: 0.17
value min 0.46724756297

In [73]:
import functools

searchspace = np.linspace(0.5, 1.5, 25)

for t in range(13, 20, 1):
    print "t is:", t
    values = []
    for tf in searchspace:

        out = pd.concat([sub.id, sub.iloc[:, 1:]
                         .applymap(functools.
                                   partial(polish,
                                           ** {"t" + str(t) + "f" : tf}))],
                        axis = 1)

        out.to_csv("../sub_polished.csv", index=False)


        subpolished = pd.read_csv("../sub_polished.csv").iloc[:, 1:]
        ll = logloss_mc(y_valid, subpolished)

        values.append(ll)
    
    print "value min", np.min(values)
    print "best tf", searchspace[np.argmin(values)]

t is: 13
value min 0.467258267743
best tf 1.125
t is: 14
value min 0.467321482246
best tf 1.08333333333
t is: 15
value min 0.467362815805
best tf 1.04166666667
t is: 16
value min 0.466944844933
best tf 1.29166666667
t is: 17
value min 0.467369589664
best tf 0.5
t is: 18
value min 0.467041311062
best tf 1.20833333333
t is: 19
value min 0.467236273787
best tf 1.125
t is: 20


TypeError: ("polish() got an unexpected keyword argument 't20f'", u'occurred at index Class_1')

In [74]:
out = pd.concat([sub.id, sub.iloc[:, 1:]
                 .applymap(functools.partial(polish,
                                             t1f=0.95,
                                             t2f=0.91,
                                             t3f=0.958,
                                             t4f=0.75,
                                             t5f=0.958,
                                             t6f=0.75))],
                axis = 1)

out.to_csv("../sub_polished.csv", index=False)


subpolished = pd.read_csv("../sub_polished.csv").iloc[:, 1:]
ll = logloss_mc(y_valid, subpolished)
print ll

0.466604701191


In [95]:
out = pd.concat([sub.id, sub.iloc[:, 1:]
                 .applymap(functools.partial(polish))],
                axis = 1)

out.to_csv("../submission.csv", index=False)


subpolished = pd.read_csv("../sub_polished.csv").iloc[:, 1:]
ll = logloss_mc(y_valid, subpolished)
print ll

0.467369589664


In [138]:
thresholds = np.linspace(0, 0.2,21)    
factors = [0.75,
0.75,
0.8,
0.9,
1,
1.05,
1.15,
1.1,
1.1,
1.1,
1.1,
1.15,
1.25,
1.25,
1.25,
1.25,
1.25,
1.15,
1.15,
1.15,
1.1]
    
out = sub
    
for threshold, factor in zip(thresholds, factors):

    out = pd.concat([out.id, out.iloc[:, 1:]
                     .applymap(functools.partial(polish_helper,
                                                threshold = threshold,
                                                delta = (thresholds[1] - thresholds[0]) / 2,
                                                factor = factor))],
                    axis = 1)

out.to_csv("../sub_niceley_polished.csv", index=False)


subpolished = pd.read_csv("../sub_niceley_polished.csv").iloc[:, 1:]
ll = logloss_mc(y_valid, subpolished)
print ll

0.47190473488


In [146]:
def polish_helper1(x, threshold, delta, factor):
    if abs(x - threshold) < delta:
        return x * factor
    else:
        return x


thresholds = np.linspace(0.8, 1, 10)    
factors = [0.1, 0.1, 0.1, 0.1, 0.1]
    
out = sub
    
for threshold, factor in zip(thresholds, factors):

    out = pd.concat([out.id, out.iloc[:, 1:]
                     .applymap(functools.partial(polish_helper1,
                                                threshold = threshold,
                                                delta = 2,
                                                factor = factor))],
                    axis = 1)

out.to_csv("../sub_niceley_polished.csv", index=False)


subpolished = pd.read_csv("../sub_niceley_polished.csv").iloc[:, 1:]
ll = logloss_mc(y_valid, subpolished)
print ll

0.467369589664


In [40]:
import functools

def polish_helper1(x, delta, factor):
    if x < delta:
        return x * factor
    else:
        return x
    
unpolished = pd.read_csv("../IPython_Submissions/master_submission.csv")

factor = 2.35

out = pd.concat([unpolished.id, unpolished.iloc[:, 1:]
                     .applymap(functools.partial(polish_helper1,
                                                delta = 0.07,
                                                factor = factor))],
                    axis = 1)

out.to_csv("../master_polished.csv", index=False)

In [41]:
unpolished.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,0.000257,0.245998,0.255722,0.492349,0.000103,0.000197,0.005170,0.000068,0.000135
1,2,0.000927,0.001515,0.000506,0.000418,0.004140,0.601189,0.001217,0.377244,0.012844
2,3,0.000002,0.000001,0.000002,0.000001,0.000000,0.999909,0.000003,0.000076,0.000005
3,4,0.000073,0.745357,0.249500,0.003799,0.000053,0.000206,0.000461,0.000233,0.000317
4,5,0.255303,0.000037,0.000024,0.000010,0.000040,0.001849,0.000646,0.021916,0.720174


In [43]:
out.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,0.000605,0.245998,0.255722,0.492349,0.000242,0.000463,0.012150,0.000159,0.000318
1,2,0.002179,0.003559,0.001188,0.000981,0.009730,0.601189,0.002860,0.377244,0.030183
2,3,0.000004,0.000003,0.000005,0.000002,0.000001,0.999909,0.000008,0.000179,0.000012
3,4,0.000172,0.745357,0.249500,0.008927,0.000125,0.000485,0.001084,0.000548,0.000745
4,5,0.255303,0.000087,0.000056,0.000023,0.000095,0.004346,0.001518,0.051503,0.720174


In [65]:
import functools

df_holdout = pd.read_csv("../data/holdout20.csv")
y_valid = df_holdout.target

def polish_helper1(x, delta):
    if x < delta:
        return np.sqrt(x)
    else:
        return x
    
unpolished = pd.read_csv("../submissions/nn-0.csv")
ll = logloss_mc(y_valid, unpolished.iloc[:, 1:])
print ll


out = pd.concat([unpolished.id, unpolished.iloc[:, 1:]
                     .applymap(functools.partial(polish_helper1,
                                                delta = 0.05))],
                    axis = 1)

out.to_csv("../afterwards.csv", index=False)

afterwards = pd.read_csv("../afterwards.csv").iloc[:, 1:]
ll = logloss_mc(y_valid, afterwards)
print ll



0.548825669935
0.592876486859
